In [ ]:
!pip install roboflow numpy ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 115.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="1CXe7ZqxLdsCrdlQeqhH")
project = rf.workspace("retina-rpndp").project("slab-3-btysl")
version = project.version(6)
dataset = version.download("yolov8")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Slab-3-6 in yolov8:: 100%|██████████| 576/576 [00:00<00:00, 875.03it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import os, glob, shutil
from pathlib import Path

# === AYARLAR ===
# Örn: "/content/New-Slab-Decoder-1" gibi dataset kökü
DATASET_ROOT = "/content/Slab-3-6"  # <- kendi yolun
LABEL_FOLDERS = ["train/labels", "valid/labels", "test/labels"]  # hangileri varsa
BACKUP_DIR = "/content/labels_backup"         # etiketleri işlem öncesi yedekle
MODE = "convert"   # "convert" -> poligonları bbox'a çevir | "drop" -> poligon satırlarını tamamen sil

def is_polygon_line(tokens):
    # tokens: ['class', 'x1', 'y1', 'x2', 'y2', ...]
    # bbox ise 5 token; polygon ise >5 (1 + 2N)
    return len(tokens) > 5

def poly_to_bbox(tokens):
    # tokens[0] = class
    # tokens[1:] = x1 y1 x2 y2 ... (normalize)
    coords = list(map(float, tokens[1:]))
    xs = coords[0::2]
    ys = coords[1::2]
    xmin, xmax = min(xs), max(xs)
    ymin, ymax = min(ys), max(ys)
    cx = (xmin + xmax) / 2.0
    cy = (ymin + ymax) / 2.0
    w  = (xmax - xmin)
    h  = (ymax - ymin)
    return [tokens[0], f"{cx:.6f}", f"{cy:.6f}", f"{w:.6f}", f"{h:.6f}"]

def audit_and_fix():
    # yedekle
    Path(BACKUP_DIR).mkdir(parents=True, exist_ok=True)
    print(f"Yedek klasörü: {BACKUP_DIR}")

    total_files = 0
    files_with_poly = 0
    total_poly_lines = 0
    converted_lines = 0
    dropped_lines = 0

    affected_files = []

    for sub in LABEL_FOLDERS:
        lbl_dir = Path(DATASET_ROOT) / sub
        if not lbl_dir.exists():
            continue

        for p in lbl_dir.glob("*.txt"):
            total_files += 1
            with open(p, "r", encoding="utf-8") as f:
                lines = [ln.strip() for ln in f if ln.strip()]

            has_poly = False
            new_lines = []
            for ln in lines:
                toks = ln.split()
                if len(toks) < 5:
                    # bozuk satır; atla
                    continue

                if is_polygon_line(toks):
                    has_poly = True
                    total_poly_lines += 1
                    if MODE == "convert":
                        # poligon -> bbox
                        try:
                            new_ln = " ".join(poly_to_bbox(toks))
                            new_lines.append(new_ln)
                            converted_lines += 1
                        except Exception:
                            # dönüştürülemedi, at
                            dropped_lines += 1
                    elif MODE == "drop":
                        # poligon satırını tamamen at
                        dropped_lines += 1
                    else:
                        # bilinmeyen mod -> eskisi gibi bırak
                        new_lines.append(ln)
                else:
                    # zaten bbox formatında
                    new_lines.append(ln)

            if has_poly:
                files_with_poly += 1
                affected_files.append(str(p))

            # dosyayı yedekle ve yaz
            if has_poly:
                # yedek
                dst = Path(BACKUP_DIR) / p.relative_to(DATASET_ROOT)
                dst.parent.mkdir(parents=True, exist_ok=True)
                shutil.copy2(p, dst)
                # yaz
                with open(p, "w", encoding="utf-8") as f:
                    f.write("\n".join(new_lines) + ("\n" if new_lines else ""))

    print("\n=== Rapor ===")
    print("Toplam label dosyası:", total_files)
    print("Poligon içeren dosya:", files_with_poly)
    print("Toplam poligon satırı:", total_poly_lines)
    print(f"İşlem modu: {MODE}")
    print("Dönüştürülen satır:", converted_lines)
    print("Silinen/atlanan satır:", dropped_lines)

    # Hangi dosyalar etkilendi listesi
    if affected_files:
        print("\nPoligon içeren dosyalar (ilk 50):")
        for p in affected_files[:50]:
            print(" -", p)
        if len(affected_files) > 50:
            print(f"... ve {len(affected_files)-50} dosya daha")

# Çalıştır
audit_and_fix()


Yedek klasörü: /content/labels_backup

=== Rapor ===
Toplam label dosyası: 282
Poligon içeren dosya: 0
Toplam poligon satırı: 0
İşlem modu: convert
Dönüştürülen satır: 0
Silinen/atlanan satır: 0


In [ ]:

# ===== Eğitim =====
from ultralytics import YOLO

# Roboflow'dan YOLOv8 formatında indirdiğin digit dataset'inin data.yaml yolu
DATA_YAML = "/content/Slab-3-6/data.yaml"  # <-- kendi yolunla değiştir

# 10 sınıf (0-9) için başlangıç ağırlığı:
model = YOLO("yolov8s.pt")  # digit için 'n' çok iyi hız/verim dengesi; istersen 's' de deneyebilirsin

model.train(
    data=DATA_YAML,
    epochs=200,        # 100 epoch
    imgsz=2048,         # digit crop'ları için uygun (160/192 de denenebilir)
    batch=-1,          # otomatik batch size
    device=0,          # GPU
    patience=20,       # erken durdurma: 20 epoch iyileşme yoksa dur
    cos_lr=True,       # cosine LR schedule
    optimizer="AdamW",   # alternatif: "AdamW"
    amp=True,          # mixed precision
    workers=8,         # dataloader worker sayısı (VRAM/CPU'ya göre 4-8)
    seed=42,
    save_period=10,    # her 10 epokta bir ağırlık kaydet (opsiyonel)
    plots=True
)

# ===== Doğrulama (opsiyonel) =====
best = YOLO("runs/detect/train/weights/best.pt")
best.val(data=DATA_YAML, imgsz=128, device=0)

# ===== Örnek tahmin (opsiyonel) =====
best.predict(source="/content/Slab-3-6/valid/images", conf=0.75, imgsz=128, save=True, device=0)


KeyboardInterrupt: 

In [ ]:
# ===================== YOLOv8 DEĞERLENDİRME: FP/FN ANALİZİ =====================
# Gereksinimler: pip install ultralytics opencv-python tqdm
# (Colab'da bir kez çalıştır: !pip -q install ultralytics opencv-python tqdm)

import os, glob, math, shutil, csv
from pathlib import Path
import cv2
import numpy as np
from tqdm import tqdm

# -------------------- KULLANICI AYARLARI (DÜZENLE) --------------------
WEIGHTS   = "/content/runs/detect/train/weights/best.pt"   # YOLOv8 ağırlığın
IMG_DIR   = "/content/Slab-3-6/test/images"          # test görüntüleri
LBL_DIR   = "/content/Slab-3-6/test/labels"          # YOLO label txt'leri
OUT_DIR   = Path("/content/eval_out")    # çıktı klasörü
CLASS_NAMES = [str(i) for i in range(10)]                    # sınıf adları (0-9)
IMG_EXTS = (".jpg",".jpeg",".png",".bmp",".webp",".tif",".tiff",".JPG",".JPEG",".PNG")
CONF_THRESH = 0.25   # model çıktı eşiği
IOU_THRESH  = 0.50   # eşleştirme IoU eşiği
SAVE_TOP_HARDEST = 40  # en zor vakalardan kaç tanesini "hard_cases" klasörüne alalım

# -------------------- MODELİ YÜKLE --------------------
from ultralytics import YOLO
model = YOLO(WEIGHTS)

OUT_DIR.mkdir(parents=True, exist_ok=True)
(OUT_DIR / "viz").mkdir(exist_ok=True)
(OUT_DIR / "hard_cases").mkdir(exist_ok=True)

# -------------------- YARDIMCI FONKSİYONLAR --------------------
def yolo_to_xyxy(box, W, H):
    # YOLO: cx, cy, w, h (normalize) -> xyxy (piksel)
    cx, cy, w, h = box
    cx, cy, w, h = cx*W, cy*H, w*W, h*H
    x1 = max(0, cx - w/2); y1 = max(0, cy - h/2)
    x2 = min(W-1, cx + w/2); y2 = min(H-1, cy + h/2)
    return [x1, y1, x2, y2]

def read_labels_txt(txt_path, W, H):
    """ YOLO txt -> list of dict: {'cls':int, 'xyxy':[x1,y1,x2,y2]} """
    out = []
    if not os.path.exists(txt_path):
        return out
    with open(txt_path, "r") as f:
        for ln in f:
            parts = ln.strip().split()
            if len(parts) < 5:
                continue
            cls = int(float(parts[0]))
            cx, cy, w, h = map(float, parts[1:5])
            xyxy = yolo_to_xyxy([cx,cy,w,h], W, H)
            out.append({"cls": cls, "xyxy": xyxy})
    return out

def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    inter_x1, inter_y1 = max(ax1,bx1), max(ay1,by1)
    inter_x2, inter_y2 = min(ax2,bx2), min(ay2,by2)
    iw, ih = max(0, inter_x2 - inter_x1), max(0, inter_y2 - inter_y1)
    inter = iw * ih
    a_area = max(0, (ax2-ax1)) * max(0, (ay2-ay1))
    b_area = max(0, (bx2-bx1)) * max(0, (by2-by1))
    union = a_area + b_area - inter + 1e-9
    return inter / union

def match_by_iou(preds, gts, iou_thr):
    """
    Sınıf eşleşmeli greedy eşleştirme.
    preds/gts: list of dict {'cls':int,'xyxy':[..], 'conf':float (preds için olabilir)}
    Dönüş: lists of indices: matched_pairs [(pi, gi)], unmatched_pred_indices, unmatched_gt_indices
    """
    if not preds and not gts:
        return [], [], []
    used_g = set()
    pairs = []

    # Her sınıf için ayrı ayrı eşleştir (daha doğru)
    classes = sorted(set([p['cls'] for p in preds] + [g['cls'] for g in gts]))
    for c in classes:
        p_idx = [i for i,p in enumerate(preds) if p['cls']==c]
        g_idx = [j for j,g in enumerate(gts)   if g['cls']==c]
        # IoU matrisini kur
        iou_mat = []
        for i in p_idx:
            row = []
            for j in g_idx:
                row.append(iou_xyxy(preds[i]['xyxy'], gts[j]['xyxy']))
            iou_mat.append(row)

        # Greedy: en yüksek IoU'lardan aşağı sırala
        cand = []
        for ii, i in enumerate(p_idx):
            for jj, j in enumerate(g_idx):
                cand.append((iou_mat[ii][jj], i, j))
        cand.sort(reverse=True, key=lambda x: x[0])

        used_p_local = set()
        used_g_local = set()
        for iou, pi, gj in cand:
            if iou < iou_thr:
                break
            if (pi not in used_p_local) and (gj not in used_g_local):
                pairs.append((pi, gj))
                used_p_local.add(pi)
                used_g_local.add(gj)

    matched_p = set([p for p,_ in pairs])
    matched_g = set([g for _,g in pairs])
    up = [i for i in range(len(preds)) if i not in matched_p]
    ug = [j for j in range(len(gts))   if j not in matched_g]
    return pairs, up, ug

def draw_boxes(img, gts, preds, pairs, up, ug, save_path):
    """
    Renk kodu:
      - GT kutuları: Yeşil (FN olan GT'ler sarı kalın)
      - Eşleşmiş TP (pred): Mavi
      - Eşleşemeyen FP (pred): Kırmızı
    Etiket: sınıf adı + skor (varsa)
    """
    H, W = img.shape[:2]
    img_vis = img.copy()

    # Önce tüm GT'leri yeşil çiz
    for i, g in enumerate(gts):
        x1,y1,x2,y2 = map(int, g["xyxy"])
        cv2.rectangle(img_vis, (x1,y1), (x2,y2), (0,255,0), 2) # GT yeşil
        label = f"GT:{CLASS_NAMES[g['cls']] if g['cls'] < len(CLASS_NAMES) else g['cls']}"
        cv2.putText(img_vis, label, (x1, max(15,y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,200,0), 2)

    # FN (eşleşemeyen GT): sarı kalın kontur
    for gi in ug:
        x1,y1,x2,y2 = map(int, gts[gi]["xyxy"])
        cv2.rectangle(img_vis, (x1,y1), (x2,y2), (0,255,255), 3)
        cv2.putText(img_vis, "FN", (x1, min(H-5,y2+15)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)

    # TP: eşleşmiş pred'ler mavi
    for pi, gi in pairs:
        pr = preds[pi]
        x1,y1,x2,y2 = map(int, pr["xyxy"])
        cv2.rectangle(img_vis, (x1,y1), (x2,y2), (255,0,0), 2)
        sc = pr.get("conf", None)
        label = f"TP:{CLASS_NAMES[pr['cls']] if pr['cls'] < len(CLASS_NAMES) else pr['cls']}"
        if sc is not None: label += f" {sc:.2f}"
        cv2.putText(img_vis, label, (x1, max(15,y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,0,0), 2)

    # FP: kırmızı
    for pi in up:
        pr = preds[pi]
        x1,y1,x2,y2 = map(int, pr["xyxy"])
        cv2.rectangle(img_vis, (x1,y1), (x2,y2), (0,0,255), 2)
        sc = pr.get("conf", None)
        label = f"FP:{CLASS_NAMES[pr['cls']] if pr['cls'] < len(CLASS_NAMES) else pr['cls']}"
        if sc is not None: label += f" {sc:.2f}"
        cv2.putText(img_vis, label, (x1, max(15,y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,200), 2)

    cv2.imwrite(str(save_path), img_vis)

def confusion_matrix_update(cm, pairs, preds, gts, up, ug, num_classes):
    # TP'ler: doğru sınıfa +1
    for pi, gi in pairs:
        c = preds[pi]['cls']
        cm[c, c] += 1
    # FP: tahmin edilen sınıfta "pred col", "GT yok" gibi saymak yerine standart CM için
    # her FP'yi "pred sınıfı, GT sınıfı = None" diye koyamayız. CM sınıflar arası dağılım için
    # basitçe FP sayısını ayrı tutacağız. (Aşağıda FP_total/FN_total ayrı tutuluyor.)
    # Eğer GT'yi yanlış sınıfa eşleşmiş gibi saymak istersek gelişmiş bir dağıtım gerekir.
    return cm

# -------------------- DEĞERLENDİRME --------------------
image_paths = []
for ext in IMG_EXTS:
    image_paths += glob.glob(os.path.join(IMG_DIR, f"*{ext}"))
image_paths = sorted(image_paths)

assert len(image_paths) > 0, f"Test klasöründe görsel bulunamadı: {IMG_DIR}"

# Sayaçlar
num_classes = max(len(CLASS_NAMES), 1)
CM = np.zeros((num_classes, num_classes), dtype=np.int64)  # yalnızca TP'ler diyagonal büyür
TP_total = 0
FP_total = 0
FN_total = 0
per_class = {i: {"TP":0,"FP":0,"FN":0} for i in range(num_classes)}

rows_csv = []
hardness_scores = []  # (score, img_path) score = FN*2 + FP (örnek)

for img_path in tqdm(image_paths, desc="Evaluating"):
    img = cv2.imread(img_path)
    if img is None:
        continue
    H, W = img.shape[:2]

    # GT oku
    base = os.path.splitext(os.path.basename(img_path))[0]
    lbl_path = os.path.join(LBL_DIR, base + ".txt")
    gts = read_labels_txt(lbl_path, W, H)

    # Tahmin
    res = model.predict(img, conf=CONF_THRESH, verbose=False)[0]
    preds = []
    if res.boxes is not None and len(res.boxes) > 0:
        xyxy = res.boxes.xyxy.cpu().numpy()
        cls   = res.boxes.cls.cpu().numpy().astype(int)
        conf  = res.boxes.conf.cpu().numpy()
        for k in range(len(cls)):
            preds.append({"cls": int(cls[k]), "xyxy": xyxy[k].tolist(), "conf": float(conf[k])})

    # Eşleştirme
    pairs, up, ug = match_by_iou(preds, gts, IOU_THRESH)

    # Sayaçlar
    TP = len(pairs)
    FP = len(up)
    FN = len(ug)

    TP_total += TP; FP_total += FP; FN_total += FN
    for pi, gi in pairs:
        c = preds[pi]['cls']
        per_class.setdefault(c, {"TP":0,"FP":0,"FN":0})
        per_class[c]["TP"] += 1
    for pi in up:
        c = preds[pi]['cls']
        per_class.setdefault(c, {"TP":0,"FP":0,"FN":0})
        per_class[c]["FP"] += 1
    for gi in ug:
        c = gts[gi]['cls'] if gts else 0
        per_class.setdefault(c, {"TP":0,"FP":0,"FN":0})
        per_class[c]["FN"] += 1

    # CM güncelle (sadece TP'leri diyagonale ekliyoruz)
    CM = confusion_matrix_update(CM, pairs, preds, gts, up, ug, num_classes)

    # Görselleştir & kaydet
    save_path = OUT_DIR / "viz" / f"{base}.jpg"
    draw_boxes(img, gts, preds, pairs, up, ug, save_path)

    # CSV satırı
    rows_csv.append({
        "image": img_path,
        "gt_count": len(gts),
        "pred_count": len(preds),
        "TP": TP, "FP": FP, "FN": FN,
        "label_file_exists": os.path.exists(lbl_path)
    })

    hardness_scores.append((FN*2 + FP, img_path))  # FN'lere iki kat ağırlık ver

# -------------------- ÖZET METRİKLER --------------------
prec = TP_total / (TP_total + FP_total + 1e-9)
rec  = TP_total / (TP_total + FN_total + 1e-9)
f1   = 2*prec*rec / (prec + rec + 1e-9)

print("\n===== GENEL ÖZET =====")
print(f"Görüntü sayısı     : {len(image_paths)}")
print(f"Toplam TP / FP / FN: {TP_total} / {FP_total} / {FN_total}")
print(f"Precision          : {prec:.4f}")
print(f"Recall             : {rec:.4f}")
print(f"F1                 : {f1:.4f}")

print("\n===== SINIF BAZLI =====")
for c in range(num_classes):
    TPc = per_class[c]["TP"]
    FPc = per_class[c]["FP"]
    FNc = per_class[c]["FN"]
    Pc = TPc / (TPc + FPc + 1e-9)
    Rc = TPc / (TPc + FNc + 1e-9)
    F1c = 2*Pc*Rc / (Pc + Rc + 1e-9)
    name = CLASS_NAMES[c] if c < len(CLASS_NAMES) else str(c)
    print(f"[{name}] TP:{TPc} FP:{FPc} FN:{FNc}  P:{Pc:.3f} R:{Rc:.3f} F1:{F1c:.3f}")

# Karışıklık matrisi (yalnızca TP diyagonal). İstersen kaydet:
np.savetxt(OUT_DIR / "confusion_matrix_tp_only.csv", CM, fmt="%d", delimiter=",")

# Per-image CSV
csv_path = OUT_DIR / "per_image_stats.csv"
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=list(rows_csv[0].keys()))
    w.writeheader()
    w.writerows(rows_csv)
print(f"\nPer-image istatistik CSV: {csv_path}")

# -------------------- ZOR VAKALARIN AYIKLANMASI --------------------
hardness_scores.sort(reverse=True, key=lambda x: x[0])
top_hard = hardness_scores[:SAVE_TOP_HARDEST]
print(f"En zor {len(top_hard)} görsel 'hard_cases/' klasörüne kopyalanıyor...")
for score, p in top_hard:
    base = os.path.basename(p)
    vis = OUT_DIR / "viz" / (os.path.splitext(base)[0] + ".jpg")
    src = vis if vis.exists() else p
    dst = OUT_DIR / "hard_cases" / os.path.basename(src)
    if src.exists():
        shutil.copy(src, dst)

print("\nBitti. Çıktılar:")
print(f"- Görsel değerlendirmeler: {OUT_DIR / 'viz'}")
print(f"- Zor vakalar:            {OUT_DIR / 'hard_cases'}")
print(f"- Confusion matrix (csv): {OUT_DIR / 'confusion_matrix_tp_only.csv'}")
print(f"- Per-image istatistik:   {csv_path}")


Evaluating: 100%|██████████| 11/11 [00:00<00:00, 13.49it/s]


===== GENEL ÖZET =====
Görüntü sayısı     : 11
Toplam TP / FP / FN: 1107 / 17 / 10
Precision          : 0.9849
Recall             : 0.9910
F1                 : 0.9880

===== SINIF BAZLI =====
[0] TP:228 FP:9 FN:0  P:0.962 R:1.000 F1:0.981
[1] TP:191 FP:2 FN:2  P:0.990 R:0.990 F1:0.990
[2] TP:245 FP:1 FN:1  P:0.996 R:0.996 F1:0.996
[3] TP:141 FP:0 FN:0  P:1.000 R:1.000 F1:1.000
[4] TP:85 FP:1 FN:0  P:0.988 R:1.000 F1:0.994
[5] TP:87 FP:0 FN:2  P:1.000 R:0.978 F1:0.989
[6] TP:39 FP:0 FN:3  P:1.000 R:0.929 F1:0.963
[7] TP:35 FP:0 FN:0  P:1.000 R:1.000 F1:1.000
[8] TP:30 FP:2 FN:0  P:0.937 R:1.000 F1:0.968
[9] TP:26 FP:2 FN:2  P:0.929 R:0.929 F1:0.929

Per-image istatistik CSV: /content/eval_out/per_image_stats.csv
En zor 11 görsel 'hard_cases/' klasörüne kopyalanıyor...

Bitti. Çıktılar:
- Görsel değerlendirmeler: /content/eval_out/viz
- Zor vakalar:            /content/eval_out/hard_cases
- Confusion matrix (csv): /content/eval_out/confusion_matrix_tp_only.csv
- Per-image istatistik:   